In [1]:
%pip install osmium folium memory_profiler

Note: you may need to restart the kernel to use updated packages.


In [2]:
import vrp
import geographs

In [3]:
Depot_coord = (18.481891, -69.913954)
qty_clients = 30
cap_trucks = 80

In [4]:
print("Loading data... ")

geoGraph = geographs.GeoGraph(limit_coords=Depot_coord)
print("Data loaded! ")

Loading data... 
Available nodes before condition: 133161
Available nodes after condition: 20812
Data loaded! 


In [5]:
Depot_node, clients, qty_trucks = vrp.create_nodes(geoGraph, Depot_coord, qty_clients, cap_trucks)

print("Depot node: ", Depot_node)
print("qty of trucks: ", qty_trucks)

Depot node:  GeoNode 4100826966 at pos (-699137564, 184818858), coords (18.4818858, -69.9137564)
qty of trucks:  4


In [6]:
qty_poblacion = 200
n_elite = 10
n_generations = 50
prob_de_mut = 0.5

solution = vrp.create_clusters(geoGraph, qty_clients, qty_trucks, cap_trucks, qty_poblacion, n_elite, n_generations, clients, prob_de_mut)

print("solution: ", solution)

solution:  [3, 2, 1, 2, 0, 2, 0, 1, 0, 3, 2, 3, 1, 3, 1, 0, 0, 1, 1, 0, 3, 0, 2, 0, 2, 1, 2, 1, 3, 2]


In [7]:
n_nearest_nodes = 3

routes, paths = vrp.create_routes(geoGraph, Depot_node, clients, qty_trucks, solution, n_nearest_nodes)

print("routes: ", routes)

for graph in routes:
    print(graph.get_nodes())


Truck 1/4
Len del client cluster: 8
Nearest nodes for client 308908454: [308923738, 308689423, 9244572806, 4100826966]
Connecting 308908454 and 308923738
Connecting 308908454 and 308689423
Connecting 308908454 and 9244572806
Connecting 308908454 and 4100826966
Nearest nodes for client 309565654: [4117721277, 9244572806, 308908454, 4100826966]
Connecting 309565654 and 4117721277
Connecting 309565654 and 9244572806
Connecting 309565654 and 308908454
Connecting 309565654 and 4100826966
Nearest nodes for client 1860279191: [308923738, 1860751308, 308908454, 4100826966]
Connecting 1860279191 and 308923738
Connecting 1860279191 and 1860751308
Connecting 1860279191 and 308908454
Connecting 1860279191 and 4100826966
Nearest nodes for client 9244572806: [308689423, 4117721277, 308908454, 4100826966]
Connecting 9244572806 and 308689423
Connecting 9244572806 and 4117721277
Connecting 9244572806 and 308908454
Connecting 9244572806 and 4100826966
Nearest nodes for client 1860751308: [1860279191, 30

In [8]:
Depot_node.name

4100826966

In [9]:
import folium
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route in paths:
    for initial_node_name in route:
        for goal_node_name in route[initial_node_name]:
            
            initial_node = routes[i].get_node_by_name(initial_node_name)
            goal_node = routes[i].get_node_by_name(goal_node_name)

            initial_marker = [initial_node.lat, initial_node.lon]
            folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
            
            goal_marker = [goal_node.lat, goal_node.lon]
            folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

            path = route[initial_node_name][goal_node_name]
            points = []

            for node in path:
                point = [node.lat, node.lon]
                points.append(point)

            folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [10]:
import genetic_algorithms_vin as gav
import random

answers = []

for route in routes:

    SUCCESS = False

    while not SUCCESS:

        graph = route.get_adj_matrix(Depot_node)

        #Params
        num_gen = 5000
        population_num = 200
        solutions_to_choose = 5
        mutation_prob = 0.9

        cities = list(range(0,len(graph)))

        initial_city = 0

        #initial population
        solutions = gav.create_population(population_num, cities, initial_city)
        #print('Propulation: ', solutions)

        best_solutions_by_gen = []

        #For each generation
        for gen in range(num_gen):

            #Calculate the total distance of each solution
            distances = gav.calc_distances_paths(graph, initial_city, solutions)
            #print("distances: ", distances)
            
            #Calculate the fitness for each solution
            fitnesess = gav.calc_fitness(distances)
            #print("fitnesess: ", fitnesess)

            if(sum(fitnesess) == 0):
                choosen_solutions = solutions[:solutions_to_choose]
            else:
                #Choose k solutions randomly based on their fitnesess
                choosen_solutions = random.choices(solutions, weights=fitnesess, k=solutions_to_choose)
            #print("choosen solutions", choosen_solutions)

            #Save the best k solutions of this generation
            best_solutions_by_gen.append(choosen_solutions)

            #Use best solutions to create a new generation by croosover and mutation (based in mutation_prob)
            crossovered_solutions = gav.crossover_solutions(choosen_solutions, population_num)
            #print("Crossovered_solutions: ", crossovered_solutions)
            solutions = gav.mutate_solutions(crossovered_solutions, mutation_prob)
            #print("Mutated_solutions: ", solutions)

        best_solution_by_gen = []

        for gen_solutions in best_solutions_by_gen:
            distances = gav.calc_distances_paths(graph, initial_city, gen_solutions)
            best_solution_of_gen = gav.sort_solutions_by_distance(gen_solutions, distances)[-1]
            best_solution_by_gen.append(best_solution_of_gen)

        final_answer, cost = sorted(best_solution_by_gen,key=lambda x: x[1])[0]

        final_answer.append(initial_city)

        print("Answer: ", final_answer, cost)

        SUCCESS = cost != float('inf')

    answers.append(final_answer)

Answer:  [0, 3, 2, 1, 6, 7, 4, 8, 5, 0] 37.53115376750807
Answer:  [0, 4, 8, 5, 6, 3, 1, 7, 2, 0] 32.035452856256356
Answer:  [0, 1, 6, 7, 5, 8, 2, 4, 3, 0] 32.91635100712162
Answer:  [0, 1, 3, 2, 4, 6, 5, 0] 19.158207474562705


In [11]:
import folium
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route_graph,route, answer in zip(routes,paths, answers):
    
    nodes_answers = [route_graph.index_to_node[answer_node] for answer_node in answer]

    print("Nodes answers: ", nodes_answers)

    j = 0
    for j in range(len(nodes_answers)-1):
        initial_node = routes[i].get_node_by_name(nodes_answers[j])
        goal_node = routes[i].get_node_by_name(nodes_answers[j+1])

        initial_marker = [initial_node.lat, initial_node.lon]
        folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
        
        goal_marker = [goal_node.lat, goal_node.lon]
        folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

        print("Initial node: ", initial_node)
        print("goal node: ", goal_node)

        path = route[initial_node.name][goal_node.name]
        points = []

        for node in path:
            point = [node.lat, node.lon]
            points.append(point)

        folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

Nodes answers:  [4100826966, 309565654, 4117721277, 9244572806, 308689423, 308908454, 308923738, 1860279191, 1860751308, 4100826966]
Initial node:  GeoNode 4100826966 at pos (-699137564, 184818858), coords (18.4818858, -69.9137564)
goal node:  GeoNode 309565654 at pos (-699279161, 184432439), coords (18.4432439, -69.9279161)
Initial node:  GeoNode 309565654 at pos (-699279161, 184432439), coords (18.4432439, -69.9279161)
goal node:  GeoNode 4117721277 at pos (-699222001, 184508725), coords (18.4508725, -69.9222001)
Initial node:  GeoNode 4117721277 at pos (-699222001, 184508725), coords (18.4508725, -69.9222001)
goal node:  GeoNode 9244572806 at pos (-699035123, 184604569), coords (18.4604569, -69.9035123)
Initial node:  GeoNode 9244572806 at pos (-699035123, 184604569), coords (18.4604569, -69.9035123)
goal node:  GeoNode 308689423 at pos (-698992953, 184752395), coords (18.4752395, -69.8992953)
Initial node:  GeoNode 308689423 at pos (-698992953, 184752395), coords (18.4752395, -69.8